In [14]:
import os
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelBinarizer

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, BatchNormalization, MaxPooling2D, Dropout, Flatten, Dense, GlobalAveragePooling2D
from tensorflow.keras.applications import MobileNetV2, ResNet50
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras.callbacks import TensorBoard


from PIL import Image

In [17]:
# Load TensorBoard to monitor training activity on realtime.

%load_ext tensorboard
%tensorboard --logdir logs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [18]:
# Load dataset images into the DataFrame.

dataset = []

for emotion in os.listdir('dataset'):
    if os.path.isdir(f'dataset/{emotion}'):
        for image_filename in os.listdir(f'dataset/{emotion}'):
            image = Image.open(f'dataset/{emotion}/{image_filename}')
            pixels = np.array(image).astype(float)
            dataset.append((pixels, emotion))

df = pd.DataFrame(dataset, columns=['pixels', 'emotion'])

In [19]:
# Normalizacion of the data.

X = np.array([MinMaxScaler().fit_transform(e) for e in df['pixels']])
Y = LabelBinarizer().fit_transform(df['emotion'])

In [20]:
# Split dataset into train and test data.

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [21]:
# Attributes for input (images width, high and channel) and output (number of labels).

_, IMAGE_WIDTH, IMAGE_HEIGHT = X.shape
_, NUM_LABELS = Y.shape

NUM_CHANNELS = 1

In [22]:
# Create a Convolutional model.

modelConv = Sequential([
    Conv2D(32, (3,3), padding="Same", activation='relu', input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, NUM_CHANNELS)),
    BatchNormalization(),
    Conv2D(32, (5,5), padding="Same", activation='relu'),
    MaxPooling2D((2,2)),
    Dropout(0.5),
    Conv2D(64, (3,3), padding="Same", activation='relu'),
    BatchNormalization(),
    Conv2D(64, (5,5), padding="Same", activation='relu'),
    MaxPooling2D((2,2)),
    Dropout(0.5),
    Conv2D(128, (3,3), padding="Same", activation='relu'),
    BatchNormalization(),
    Conv2D(128, (5,5), padding="Same", activation='relu'),
    MaxPooling2D((2,2)),
    Dropout(0.5),
    Flatten(),
    Dense(256, activation='relu'),
    Dense(NUM_LABELS, activation='softmax'),
])

modelConv.compile(loss='categorical_crossentropy', optimizer=Adam(0.001), metrics=['accuracy'])

In [23]:
# Train the convolutional model.

modelConv.fit(X_train, Y_train, batch_size=32, epochs=50, validation_data=(X_test, Y_test), callbacks=[TensorBoard(log_dir="logs/modelConv")])

Epoch 1/50


2022-12-29 01:38:18.231328: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


898/898 [==============================] - ETA: 0s - loss: 1.7355 - accuracy: 0.3247

2022-12-29 01:38:41.594965: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


898/898 [==============================] - 26s 28ms/step - loss: 1.7355 - accuracy: 0.3247 - val_loss: 1.8935 - val_accuracy: 0.3564
Epoch 2/50
898/898 [==============================] - 25s 28ms/step - loss: 1.4287 - accuracy: 0.4424 - val_loss: 1.3798 - val_accuracy: 0.4510
Epoch 3/50
898/898 [==============================] - 25s 28ms/step - loss: 1.3313 - accuracy: 0.4829 - val_loss: 1.3112 - val_accuracy: 0.4994
Epoch 4/50
898/898 [==============================] - 25s 28ms/step - loss: 1.2672 - accuracy: 0.5045 - val_loss: 1.2880 - val_accuracy: 0.5067
Epoch 5/50
898/898 [==============================] - 25s 27ms/step - loss: 1.2236 - accuracy: 0.5312 - val_loss: 1.1991 - val_accuracy: 0.5414
Epoch 6/50
898/898 [==============================] - 25s 27ms/step - loss: 1.1702 - accuracy: 0.5561 - val_loss: 1.1575 - val_accuracy: 0.5557
Epoch 7/50
898/898 [==============================] - 25s 28ms/step - loss: 1.1345 - accuracy: 0.5700 - val_loss: 1.1339 - val_accuracy: 0.5791
Epo

In [ ]:
score, acc = modelConv.evaluate(X_test, Y_test, batch_size=100)
print('Test score:', score)
print("Test accuracy:", acc)

In [3]:
# Load dataset images into the DataFrame and convert into RGB (required by MobileNetV2)

dataset = []

for emotion in os.listdir('dataset'):
    if os.path.isdir(f'dataset/{emotion}'):
        for image_filename in os.listdir(f'dataset/{emotion}'):
            image = Image.open(f'dataset/{emotion}/{image_filename}').convert("RGB")
            pixels = np.array(image).astype(float)
            dataset.append((pixels, emotion))

df = pd.DataFrame(dataset, columns=['pixels', 'emotion'])

In [4]:
# Normalizacion of the data.

X = np.array((df['pixels']/255).values.tolist())
Y = LabelBinarizer().fit_transform(df['emotion'])

In [5]:
# Split dataset into train and test data.

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [6]:
# Attributes for input (images width, high and channel) and output (number of labels).

_, IMAGE_WIDTH, IMAGE_HEIGHT, NUM_CHANNELS = X.shape
_, NUM_LABELS = Y.shape

In [7]:
# Create a MobileNetV2 based model.

base_model = MobileNetV2(input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, NUM_CHANNELS),
                         include_top=False,
                         weights='imagenet')

#base_model.trainable = False

modelMobNet = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dropout(0.3),
    Dense(128, activation='softmax'),
    Dense(NUM_LABELS, activation='softmax'),
])

modelMobNet.compile(loss='categorical_crossentropy', optimizer=Adam(0.001), metrics=['accuracy'])

Metal device set to: Apple M1 Pro


2022-12-28 23:10:05.810564: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-12-28 23:10:05.810586: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [12]:
# Train the MobileNetV2 based model.

modelMobNet.fit(X_train, Y_train, batch_size=32, epochs=50,  validation_data=(X_test, Y_test), callbacks=[TensorBoard(log_dir="logs/modelMobNet")])

Epoch 1/25
898/898 [==============================] - 46s 51ms/step - loss: 0.6397 - accuracy: 0.7833 - val_loss: 1.4007 - val_accuracy: 0.5747
Epoch 2/25
898/898 [==============================] - 45s 50ms/step - loss: 0.6121 - accuracy: 0.7934 - val_loss: 1.4480 - val_accuracy: 0.5626
Epoch 3/25
898/898 [==============================] - 47s 52ms/step - loss: 0.5966 - accuracy: 0.8006 - val_loss: 1.4254 - val_accuracy: 0.5626
Epoch 4/25
898/898 [==============================] - 47s 52ms/step - loss: 0.5802 - accuracy: 0.8056 - val_loss: 1.3481 - val_accuracy: 0.5795
Epoch 5/25
898/898 [==============================] - 47s 52ms/step - loss: 0.5753 - accuracy: 0.8073 - val_loss: 1.4000 - val_accuracy: 0.5678
Epoch 6/25
898/898 [==============================] - 46s 52ms/step - loss: 0.5666 - accuracy: 0.8120 - val_loss: 1.4512 - val_accuracy: 0.5660
Epoch 7/25
898/898 [==============================] - 46s 51ms/step - loss: 0.5224 - accuracy: 0.8270 - val_loss: 1.4646 - val_accuracy:

KeyboardInterrupt: 

In [11]:
score, acc = modelMobNet.evaluate(X_test, Y_test, batch_size=100)
print('Test score:', score)
print("Test accuracy:", acc)

72/72 [==============================] - 1s 19ms/step - loss: 1.3364 - accuracy: 0.5678
Test score: 1.3364269733428955
Test accuracy: 0.5678462386131287


In [15]:
# Create a ResNet50 based model.

base_model = model = ResNet50(input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, NUM_CHANNELS),
                                               include_top=False,
                                               weights='imagenet')

#base_model.trainable = False

modelResNet = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dropout(0.3),
    Dense(128, activation='softmax'),
    Dense(NUM_LABELS, activation='softmax'),
])

modelResNet.compile(loss='categorical_crossentropy', optimizer=Adam(0.001), metrics=['accuracy'])

94765736/94765736 [==============================] - 2s 0us/step


In [16]:
# Train the ResNet50 based model.

modelResNet.fit(X_train, Y_train, batch_size=32, epochs=50,  validation_data=(X_test, Y_test), callbacks=[TensorBoard(log_dir="logs/modelResNet")])

Epoch 1/50


2022-12-29 00:09:21.288420: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


898/898 [==============================] - ETA: 0s - loss: 1.8366 - accuracy: 0.2484

2022-12-29 00:10:30.261837: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


898/898 [==============================] - 77s 79ms/step - loss: 1.8366 - accuracy: 0.2484 - val_loss: 1.8119 - val_accuracy: 0.2552
Epoch 2/50
898/898 [==============================] - 68s 76ms/step - loss: 1.7841 - accuracy: 0.2579 - val_loss: 1.7410 - val_accuracy: 0.3013
Epoch 3/50
898/898 [==============================] - 68s 76ms/step - loss: 1.7220 - accuracy: 0.3154 - val_loss: 1.7116 - val_accuracy: 0.3314
Epoch 4/50
898/898 [==============================] - 68s 76ms/step - loss: 1.6258 - accuracy: 0.3744 - val_loss: 1.5782 - val_accuracy: 0.4117
Epoch 5/50
898/898 [==============================] - 68s 76ms/step - loss: 1.5190 - accuracy: 0.4291 - val_loss: 1.4687 - val_accuracy: 0.4490
Epoch 6/50
898/898 [==============================] - 67s 75ms/step - loss: 1.4464 - accuracy: 0.4595 - val_loss: 1.4089 - val_accuracy: 0.4723
Epoch 7/50
898/898 [==============================] - 67s 75ms/step - loss: 1.3820 - accuracy: 0.4863 - val_loss: 1.8613 - val_accuracy: 0.2455
Epo